## In this notebook:

* 3 input dataframe: chr18, 19, 20
* using all numeric scores for predictors
* fully connected neural net
* training with dataloading and epochs
* aucuracy assessement of .954 on held-out test 

### Imports

In [71]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [72]:
#################################################### 
## pull in pickled dataframe:
df = pd.read_pickle("./pickled_df/all_scores_chr18,20.pkl")


# Split our data into two separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                     SIFT_score SIFT4G_score Polyphen2_HDIV_score  \
chr pos      ref alt                                                
20  63408436 G   A            0          0.0                    0   
    63408468 G   A            0        0.382                    0   
    63408484 T   A            0            0                    0   
    63408517 G   A            0        0.002                    0   
    63408524 G   C            0        0.002                    0   

                     Polyphen2_HVAR_score LRT_score MutationTaster_score  \
chr pos      ref alt                                                       
20  63408436 G   A                      0  0.000016              0.99997   
    63408468 G   A                      0  0.040548              0.97061   
    63408484 T   A                      0  0.114895                    1   
    63408517 G   A                      0  0.744725             0.998933   
    63408524 G   C                      0  0.663984         

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [73]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [74]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(27, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 27))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [75]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [0/1050 (0%)]	Loss: 0.693924
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [59/1050 (6%)]	Loss: 0.690112
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [118/1050 (11%)]	Loss: 0.684353
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [177/1050 (17%)]	Loss: 0.690112
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [236/1050 (22%)]	Loss: 0.671701
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [295/1050 (28%)]	Loss: 0.661779
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [354/1050 (33%)]	Loss: 0.643532
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [413/1050 (39%)]	Loss: 0.637452
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [472/1050 (44%)]	Loss: 0.622531
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [531/1050 (50%)]	Loss: 0.637350
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 1/20 [590/1050 (56%)]	Loss: 0.610851
torch.Size([59, 27])
torch

Predictions: tensor([1, 1, 0,  ..., 1, 1, 1])
Actual values: tensor([1., 1., 0.,  ..., 1., 1., 1.])
 Train Epoch: 4/20 [1050/1050 (94%)]	Loss: 0.420035	 Validation Accuracy: 86.29%
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [0/1050 (0%)]	Loss: 0.391985
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [59/1050 (6%)]	Loss: 0.487584
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [118/1050 (11%)]	Loss: 0.422534
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [177/1050 (17%)]	Loss: 0.443984
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [236/1050 (22%)]	Loss: 0.381445
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [295/1050 (28%)]	Loss: 0.390877
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [354/1050 (33%)]	Loss: 0.399789
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [413/1050 (39%)]	Loss: 0.366093
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 5/20 [472/1050 (44%)]	Loss: 0.404420
torch.Size([59, 27])
to

 Train Epoch: 8/20 [799/1050 (94%)]	Loss: 0.378564
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.7248, 0.6453, 0.6356],
        [0.0000, 0.0000, 0.0000,  ..., 0.7025, 0.0000, 0.7144],
        [0.0080, 0.0130, 0.5320,  ..., 0.6979, 0.5705, 0.7144],
        ...,
        [0.0000, 0.1150, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.0606, 0.0742, 0.0792]])
Predictions: tensor([1, 1, 0,  ..., 1, 1, 1])
Actual values: tensor([1., 1., 0.,  ..., 1., 1., 1.])
 Train Epoch: 8/20 [1050/1050 (94%)]	Loss: 0.378564	 Validation Accuracy: 91.71%
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 9/20 [0/1050 (0%)]	Loss: 0.360363
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 9/20 [59/1050 (6%)]	Loss: 0.347389
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 9/20 [118/1050 (11%)]	Loss: 0.353931
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 9/20 [177/1050 (17%)]	Loss: 0.410300
torch.Size([59, 27]

 Train Epoch: 12/20 [649/1050 (61%)]	Loss: 0.433497
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [708/1050 (67%)]	Loss: 0.477248
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [767/1050 (72%)]	Loss: 0.470479
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [826/1050 (78%)]	Loss: 0.424275
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [885/1050 (83%)]	Loss: 0.414919
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 12/20 [944/1050 (89%)]	Loss: 0.394465
torch.Size([47, 27])
torch.Size([47])
 Train Epoch: 12/20 [799/1050 (94%)]	Loss: 0.397326
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.7248, 0.6453, 0.6356],
        [0.0000, 0.0000, 0.0000,  ..., 0.7025, 0.0000, 0.7144],
        [0.0080, 0.0130, 0.5320,  ..., 0.6979, 0.5705, 0.7144],
        ...,
        [0.0000, 0.1150, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.5473, 0.6453, 0.5628],
        [0.0000, 0.0000, 0.0000,  ..., 0.0606, 0.0742, 0.0792]])
Prediction

 Train Epoch: 16/20 [295/1050 (28%)]	Loss: 0.336830
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [354/1050 (33%)]	Loss: 0.459900
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [413/1050 (39%)]	Loss: 0.379647
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [472/1050 (44%)]	Loss: 0.401913
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [531/1050 (50%)]	Loss: 0.356904
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [590/1050 (56%)]	Loss: 0.395936
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [649/1050 (61%)]	Loss: 0.383074
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [708/1050 (67%)]	Loss: 0.396905
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [767/1050 (72%)]	Loss: 0.400516
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [826/1050 (78%)]	Loss: 0.319217
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 16/20 [885/1050 (83%)]	Loss: 0.413244
torch.Size([59, 27])
torch.Size([59])
 Train Epo

 Train Epoch: 20/20 [0/1050 (0%)]	Loss: 0.398456
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [59/1050 (6%)]	Loss: 0.375376
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [118/1050 (11%)]	Loss: 0.330956
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [177/1050 (17%)]	Loss: 0.444653
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [236/1050 (22%)]	Loss: 0.332821
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [295/1050 (28%)]	Loss: 0.395402
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [354/1050 (33%)]	Loss: 0.437810
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [413/1050 (39%)]	Loss: 0.330967
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [472/1050 (44%)]	Loss: 0.415320
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [531/1050 (50%)]	Loss: 0.410916
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 20/20 [590/1050 (56%)]	Loss: 0.403977
torch.Size([59, 27])
torch.Size([59])
 Train Epoch: 2

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [76]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]

y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))


 Testing Accuracy: 95.44%
